# LR Parser

## What is

> In computer science, LR parsers are a type of bottom-up parser that analyse deterministic context-free languages in linear time. – Wikipedia

Yeah, that makes no sense to you just as it is to me.

So in this notebook, I will try to explain what an LR parser is in my own understanding, starting from smaller concepts and working our way up to the somewhat complex idea of the parser as a whole (much like the bottom-up approach of LR parser itself). And we will write an LR parser in Python together.

But I must first admit, that much of my own understanding of LR parser comes from the Wikipedia cited above, and I have zero academic reference whatsoever. There is no guarantee to the theoretical rigour of any of the content below. This is, again just my personal train of thought derailed in front of you.

Let's begin.

Oh before we begin, let's just do some Python imports here, since there's really no better place to put them in this notebook. There is no need to worry if you don't know some of the imported stuff. It's only relevent to our implementation, mostly typing, and has nothing to do with the notion of LR parser itself.

In [1]:
from collections import deque
from pprint import pprint
from string import ascii_uppercase
from typing import NamedTuple, Optional

from ordered_set import OrderedSet
from typing_extensions import Self

## Token

Tokens are words, or individual components found in your source code. The following Python source code:
```python
def my_function():
    pass
```
corresponds to 11 tokens:
```
'def' 'my_function' '(' ')' ':' 'NEWLINE' 'INDENT' 'pass' 'NEWLINE' 'DEDENT' 'END'
```

Here I used single quotes to show you each token, but this is no standard and is simply for clarity. Notice some tokens are not easily visible from the source code, such as NEWLINE, INDENT, DEDENT, and END.

You can verify this using the `tokenize` module from Python standard library.



In [2]:
import io
from tokenize import generate_tokens, tok_name

source_code = """\
def my_function():
    pass
"""
for token in generate_tokens(io.StringIO(source_code).readline):
    print(tok_name[token.type], repr(token.string))

NAME 'def'
NAME 'my_function'
OP '('
OP ')'
OP ':'
NEWLINE '\n'
INDENT '    '
NAME 'pass'
NEWLINE '\n'
DEDENT ''
ENDMARKER ''


The job of an LR parser (or any parser) is to take a stream of tokens as input, and output an Abstract Syntax Tree.
```python
def lr_parser(tokens: List[Token]) -> ASTree:
```

Well, OK, when I said tokens are words, I lied a bit. Here we are going to slightly overload the concept of a token. Tokens are not just the words you saw above, but also larger structures they make up of. So the source code `1 + 1` is not just three tokens, but can also be seen as 1 `Add` token, which is in turn an expression, or `Expr` token. These are shown in the following tree structure, known as an Abstract Syntax Tree.
```
Expr
└─ Add
   ├─ '1'
   ├─ '+'
   └─ '1'
```

You can probably feel that '1' and Expr are quite different: '1' is very concrete and Expr is more abstract. In fact, we call the first type of tokens **Terminals**, and the second type **Non-Terminals**.

### Terminals
Terminal tokens are single tokens that you can find from the token stream. They are called Terminals because they will become the leaf nodes of your tree. I will denote Terminal tokens with single quotes. The following are five Terminals:
```python
'def'   # the keyword return
'3.14'  # the number 3.14
'+'     # the plus sign
'('     # the left parenthesis
'"abc"' # the string literal "abc"
```

### Non-Terminals
Non-Terminal tokens are the non-leaf nodes in your tree. I will denote Non-Terminal tokens with a word starting with an uppercase letter. These are four Non-Terminals:
```python
Stmt # a statement
Expr # an expression
Add  # an add expression, such as 1 + 2
Atom # an atomic expression, such as an identifier or a number
```

Now that we have seen the concept of tokens, let's write them in Python.

Here I use `NamedTuple` to make a token immutable and hashable, which will come in handy later. I also override the `__repr__` functions to make them look nicer when printed, but don't worry about these.

The `make_token` function helps us to create a token by giving it a string in the notation I used above: either in single quotes or starting with an upper case letter.

In [3]:
class Token(NamedTuple):
    name: str


class Term(Token):  # Terminals
    def __repr__(self) -> str:
        return "Term(" + self.name + ")"


class NTerm(Token):  # Non-Terminals
    def __repr__(self) -> str:
        return "NTerm(" + self.name + ")"


def make_token(token: str) -> Token:
    if token[0] in ascii_uppercase:
        return NTerm(token)
    elif token[0] == "'":
        return Term(token)
    else:
        raise ValueError

print(make_token("'3.14'"), make_token("'+'"), make_token("Expr"))

Term('3.14') Term('+') NTerm(Expr)


## Rule

**Rules** are where we define our syntax, or grammar. A Rule starts with a Non-Terminal token, followed by a sequence of Terminal and Non-Terminal tokens.

For example, we can make a rule that an Atom is made of either a '0' or a '1'.
```
Atom <= '0'
Atom <= '1'
```

We’ll call the Non-Terminal token to the left of `<=` the Left-Hand-Side of a Rule, or LHS. Similarly, the sequence of mixed tokens to the right of `<=`  is called the Right-Hand-Side of a Rule, or RHS.

Rules can be recursive. In fact, one of the major advantage of LR parsers is the ability to handle _Left Recursion_, but I won't go into detail here.

For example, an Expr can be made of another Expr, a plus sign, and an Atom.
```
Expr <= Expr '+' Atom
```

Again, let's write this in Python. And we'll also write a `make_rule` function to help us create a rule from a grammar string like above.

In [4]:
class Rule(NamedTuple):
    left: Token
    right: tuple[Token]

    def __repr__(self) -> str:
        return "Rule: " + str(self.left) + " <= " + " ".join(map(str, self.right))


def make_rule(grammar: str) -> Rule:
    left, right = grammar.split(" <= ")

    left = make_token(left)
    assert isinstance(left, NTerm)

    right = tuple(make_token(token) for token in right.split())

    return Rule(left, right)

print(make_rule("Expr <= Expr '+' Atom"))

Rule: NTerm(Expr) <= NTerm(Expr) Term('+') NTerm(Atom)


## Items and Item Sets

I believe so far things have been straightforward. But it's gonna start to get a bit confusing from here.

A Rule can generate many **LR(0) Items**, or simply **Items**, by placing a "dot" at different positions of the RHS of the Rule. For clarity, I will represent the "dot" with the exponent or XOR sign `^`.

The Rule above corresponds to 4 Items:
```
Expr <= ^ Expr '+' Atom
Expr <= Expr ^ '+' Atom
Expr <= Expr '+' ^ Atom
Expr <= Expr '+' Atom ^
```

Imagine the "dot" is a cursor that moves rightwards as you parse through the Rule. So for a Rule with `n` tokens on its RHS, there are `n+1` positions for the "dot", and therefore `n+1` Items.

Once you understood what an Item is, an Item Set is much simpler to understand. It is simply a set of Items. Note that these Items can be made from different rules.

When implementing this in Python, we'll need the ability to move the "dot" to the right by 1 token, and to know what's immediately after the "dot", so we write these as two methods of the Item class. The ItemSet class subclasses `tuple[Item]`, but we'll also use `list[Item]` to represent an Item Set, depending on the mutability we need.

We also make a function `make_item_set`, but this function is actually not used in our final parser. It is just to show what an Item Set made from a single rule looks like.

In [6]:
class Item(NamedTuple):
    base: Rule
    dot: int

    def move_right(self) -> Self:
        return Item(base=self.base, dot=self.dot + 1)

    def right_of_dot(self) -> Optional[Token]:
        if self.dot < len(self.base.right):
            return self.base.right[self.dot]
        else:
            return None


class ItemSet(tuple[Item]):
    pass


def make_item_set(rule: Rule) -> ItemSet:
    items = (Item(base=rule, dot=dot) for dot in range(len(rule.right) + 1))
    return ItemSet(items)


pprint(make_item_set(make_rule("Expr <= Expr '+' Atom")))

(Item(base=Rule: NTerm(Expr) <= NTerm(Expr) Term('+') NTerm(Atom), dot=0),
 Item(base=Rule: NTerm(Expr) <= NTerm(Expr) Term('+') NTerm(Atom), dot=1),
 Item(base=Rule: NTerm(Expr) <= NTerm(Expr) Term('+') NTerm(Atom), dot=2),
 Item(base=Rule: NTerm(Expr) <= NTerm(Expr) Term('+') NTerm(Atom), dot=3))


## Closure and State

Item Set is an important concept in LR parsers, because it represents a **State** that our parser is in, if you imagine our parser as a _finite automaton_. But not all Item Sets represent a possible and reachable State in our parsing process. The Item Sets that do represent a State are called a closed Item Set, or a **Closure**.

And thus, we can equte a State to a Closure, or a closed Item Set. We'll mostly be using the word State from here, but just know that these terms are essentially equivalent.

From any Item Set, we can _close_ it by the following steps:
1. Gather all the Non-Terminals that are immediately after a "dot" from all Items in the Item Set.
1. For all rules starting with these Non-Terminals, add an Item to our Item Set, with "dot" all set at 0.
1. Repeat until there is no new Items to be added.

For example, if Non-Terminal `X` is after a dot in the existing set, and there is a rule `X <= Y Z`, then add `X <= ^ Y Z` to the set.

The Python implementation of this might take some time to understand, so please take your time.

In [ ]:
class State(ItemSet):
    pass


def make_closure(rules: tuple[Rule], items: list[Item]) -> State:
    non_terminals = deque(
        filter(
            lambda token: isinstance(token, NTerm),
            [item.right_of_dot() for item in items],
        )
    )
    seen = set()
    while non_terminals:
        non_term = non_terminals.popleft()
        if non_term not in seen:
            for rule in rules:
                if rule.left == non_term:
                    new_item = Item(base=rule, dot=0)
                    if isinstance(new_item.right_of_dot(), NTerm):
                        non_terminals.append(new_item.right_of_dot())
                    items.append(new_item)
        seen.add(non_term)

    return State(items)

## Parsing Table